In [12]:
from json import load
import pandas as pd

from collections import defaultdict

In [13]:
with open("../data/tenders/tenders_sber_22_19.json") as f:
    data = load(f)

In [14]:
# решил сохранять суммы выигранных тендеров, и где просто участвовал, тк так просто посчитать и аггрегации по суммам, и количество
inn2stats = defaultdict(
    lambda: {"participated_amounts": [], "won_amounts": []}
)

In [15]:
for sample in data:
    amount = sample["amount"]
    for inn, inn_data in sample["players"].items():
        if inn_data["dopusk"] == False:
            continue
        if inn_data["zakluchen dogovor"]:
            inn2stats[inn]["won_amounts"].append(amount)
        inn2stats[inn]["participated_amounts"].append(amount)

In [16]:
def mean(values):
    if len(values) == 0:
        return None
    else:
        return sum(values) / len(values)

In [17]:
inn2features = {}

for inn, stats in inn2stats.items():
    features = {
        "cnt_participated": len(stats["participated_amounts"]),
        "cnt_won": len(stats["won_amounts"]),
        
        "sum_participated_amount": sum(stats["participated_amounts"]),
        "sum_won_amount": sum(stats["won_amounts"]),
        
        "mean_participated_amount": mean(stats["participated_amounts"]),
        "mean_won_amount": mean(stats["won_amounts"]),
    }
    inn2features[inn] = features

In [18]:
aggregates_df = pd.DataFrame.from_records(inn2features).T

In [19]:
aggregates_df.index.name = "inn"

In [20]:
aggregates_df.to_csv("../data/tenders_sber_22_19_aggregates.csv")